In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical

def run_network(network):
    nb_classes = 10
    batch_size = 256
    input_shape = (784,)

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)
    
    nb_layers = network['nb_layers']
    nb_neurons = network['nb_neurons']
    activation = network['activation']
    optimizer = network['optimizer']
    model = Sequential()
    for layer in range(nb_layers):
        if layer == 0:
            model.add(Dense(nb_neurons, activation=activation, input_shape=input_shape))
        else:
            model.add(Dense(nb_neurons, activation = activation))
        model.add(Dropout(0.1))
    model.add(Dense(nb_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=20,
              verbose=0,
              validation_data=(x_test, y_test))

    #zbieranie wyników
    score = model.evaluate(x_test, y_test, verbose=0)

    return score[1]

In [ ]:
from functools import reduce
from operator import add
import random

import random

class Network():
    def __init__(self, nn_param_choices=None):
        self.accuracy = 0.
        self.nn_param_choices = nn_param_choices
        self.network = {}

    def create_random(self):
        for key in self.nn_param_choices:
            self.network[key] = random.choice(self.nn_param_choices[key])

    def create_set(self, network):
        self.network = network

    def train(self):
        if self.accuracy == 0.:
            self.accuracy = run_network(self.network)

    def print_network(self):
        print(self.network)
        print(f'Network accuracy: {self.accuracy * 100}%')

In [3]:
MUTATE_CHANCE = 0.15
RANDOM_SELECT = 0.15
RETAIN = 0.5
PARAMS = {
    'nb_neurons': [16, 32, 64, 128, 256],
    'nb_layers': [1, 2, 3],
    'activation': ['relu', 'tanh','selu', 'sigmoid'],
    'optimizer': ['adam', 'sgd', 'adadelta', 'adamax'],
}

def create_population(count):
    pop = []
    for _ in range(0, count):
        network = Network(PARAMS)
        network.create_random()
        pop.append(network)
    return pop

def fitness(network):
    return network.accuracy

def grade(pop):
    summed = reduce(add, (fitness(network) for network in pop))
    return summed / float((len(pop))) 

def mutate(network):
    mutation = random.choice(list(PARAMS.keys()))
    network.network[mutation] = random.choice(PARAMS[mutation])
    return network

def cross(parent1, parent2):
    children = []
    for i in range(2):
        child = {}
        for param in PARAMS:
            child[param] = random.choice(
                [parent1.network[param], parent2.network[param]]
            )

        network = Network(PARAMS)
        network.create_set(child)

        if MUTATE_CHANCE > random.random():
            network = mutate(network)

        children.append(network)

    return children

def evolve(pop):
    graded = [(fitness(network), network) for network in pop]

    graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=True)]

    retain_length = int(len(graded)*RETAIN)

    parents = graded[:retain_length]

    for individual in graded[retain_length:]:
        if RANDOM_SELECT > random.random():
            parents.append(individual)

    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []

    while len(children) < desired_length:
        parent1 = random.randint(0, parents_length-1)
        parent2 = random.randint(0, parents_length-1)

        if parent1 != parent2:
            parent1 = parents[parent1]
            parent2 = parents[parent2]

            descendants = cross(parent1, parent2)

            for descendant in descendants:
                if len(children) < desired_length:
                    children.append(descendant)

    parents.extend(children)

    return parents

In [4]:
def print_networks(networks):
    print('_'*100)
    for network in networks:
        network.print_network()

In [5]:
def get_average_accuracy(networks):
    total_accuracy = 0
    for network in networks:
        total_accuracy += network.accuracy

    return total_accuracy / len(networks)

In [ ]:
generations = 6
population = 2

print(f'Run with {generations} generations and {population} populations')

networks = create_population(population)

for i in range(generations):
    print(f'Generartion {i+1}...')
    for idx, network in enumerate(networks):
        print(f'Network {idx}')
        network.train()
    average_accuracy = get_average_accuracy(networks)
    print(f'Average for generation {i+1}: {average_accuracy * 100}%')

    if i != generations - 1:
        networks = evolve(networks)

networks = sorted(networks, key=lambda x: x.accuracy, reverse=True)

print_networks(networks[:10])


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
import numpy as np

parameters = {'solver': ['adam', 'sgd', 'adadelta', 'adamax'], 'max_iter': [20],
              'hidden_layer_sizes': [(16,), (32,), (64,), (128,), (256,), (16, 16,), (32, 32,), (64, 64,), (128, 128,), (256, 256,), (16, 16, 16,), (32, 32, 32,), (64, 64, 64,), (128, 128, 128,), (256, 256, 256,), ],
                'batch_size': [256], 'activation':  ['relu', 'tanh','selu', 'sigmoid'], }

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

grid_search = GridSearchCV(MLPRegressor(), parameters,
                           n_jobs=-1, verbose=1, cv=4)

grid_search.fit(x_train, y_train)

print(grid_search.score(x_test, y_test))
print(grid_search.best_params_)
